In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models.forecasting.random_forest import RandomForest
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Visualization settings
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

In [2]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

DATA_FILE_PATH = "../data/ground_station_clean.csv"
df = pd.read_csv(DATA_FILE_PATH)

In [3]:
encoders = {
    "cyclic": {
        "past": ["month", "dayofyear", "day", "hour", "minute"],
        "future": ["month", "dayofyear", "day", "hour", "minute"]
    },
    "transformer": Scaler(),
    "datetime_attribute": {
        "past": ["year"],
        "future": ["year"]
    }
}


In [4]:
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Create directory to save iteration results
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [5]:
def objective(trial):
    num_target_lags = trial.suggest_int('num_target_lags', 5, 24)
    target_lags_list = [-i for i in range(1, num_target_lags + 1)]
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 800, step=50),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'criterion': trial.suggest_categorical('criterion', ['squared_error', 'poisson']),
        'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 0.0, 0.5),
        'n_jobs': -1
    }

    n = int(len(df) * 0.8)
    train_df_fold, val_df_fold = df.iloc[:n], df.iloc[n:]
    train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
    val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

    print(f"\nStarting Trial {trial.number}")
    print(f"Hyperparameters: {trial.params}")
    print("\nTraining the model...")
    print(f"Train set: {len(train_fold)} samples")
    print(f"Validation set: {len(val_fold)} samples")

    scaler = Scaler()
    scaler = scaler.fit(train_fold)
    train_scaled = scaler.transform(train_fold)
    val_scaled = scaler.transform(val_fold)

    _work_dir = "/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/"
    _model_name = "model_optuna_temp"
    os.makedirs(_work_dir, exist_ok=True)

    model = RandomForest(
        lags=target_lags_list,
        add_encoders=encoders,
        **params
    )

    model.fit(
        series=train_scaled,
        val_series=val_scaled,
    )

    forecasts = model.historical_forecasts(
        val_scaled,
        forecast_horizon=1,
        stride=1,
        retrain=False,
    )

    forecasts_t = scaler.inverse_transform(forecasts)
    s = scaler.inverse_transform(val_scaled)
    metrics = {}
    print("Starting time series verification for consistency...")
    try:
        for target in target_columns:
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),
                'RMSE': rmse(s[target], forecasts_t[target]),
                'MAE': mae(s[target], forecasts_t[target]),
                'R2': r2_score(s[target], forecasts_t[target]),
                'SMAPE': smape(s[target], forecasts_t[target]),
            }
        metrics_df = pd.DataFrame(metrics).T
        print("\nPerformance metrics:")
        print(metrics_df)
    except Exception as e:
        print(e)

    overall_mae_val = mae(val_scaled, forecasts)
    print(f"The MAE for this fold was {overall_mae_val}")

    trial_dict = {
        "trial_number": trial.number,
        "overall_mae_val": overall_mae_val,
        "params": params,
        "metrics": metrics_df.to_dict(),
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Results of trial {trial.number} saved in {json_path}")
    return overall_mae_val if not np.isnan(overall_mae_val) else float("inf")


In [6]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [7]:
study = optuna.create_study(direction="minimize")
num_hyperparams = 9
n_trials = 7 * num_hyperparams
study.optimize(objective, n_trials=n_trials, callbacks=[print_callback])

[I 2025-05-27 21:33:11,338] A new study created in memory with name: no-name-f52ecee0-27ca-4c04-852b-56aaeb301d3a
Specified past encoders in `add_encoders` at model creation but model does not accept past covariates. past encoders will be ignored.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.



Starting Trial 0
Hyperparameters: {'num_target_lags': 24, 'n_estimators': 650, 'max_depth': 18, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 0.5204563904175212, 'bootstrap': True, 'criterion': 'poisson', 'min_weight_fraction_leaf': 0.052310916507274985}

Training the model...
Train set: 6647 samples
Validation set: 1662 samples


[I 2025-05-27 21:33:24,193] Trial 0 finished with value: 0.17478106779814798 and parameters: {'num_target_lags': 24, 'n_estimators': 650, 'max_depth': 18, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 0.5204563904175212, 'bootstrap': True, 'criterion': 'poisson', 'min_weight_fraction_leaf': 0.052310916507274985}. Best is trial 0 with value: 0.17478106779814798.
Specified past encoders in `add_encoders` at model creation but model does not accept past covariates. past encoders will be ignored.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.


Starting time series verification for consistency...

Performance metrics:
                                    MSE         RMSE          MAE         R2  \
Temperature                3.090529e+01     5.559253     4.557309  -0.012204   
Precipitation_accumulated  5.561448e+01     7.457511     7.221308 -15.081717   
Humidity                   1.507979e+02    12.279981    10.793980  -0.028739   
Wind_Speed_kmh             2.072097e+00     1.439478     1.092453  -0.533807   
Soil_Moisture              2.240923e+06  1496.971226  1476.175178 -37.539993   
Soil_Temperature           1.667386e+01     4.083363     3.345842  -0.030637   
Wind_Dir_Sin               6.850217e-01     0.827660     0.650084  -0.246267   
Wind_Dir_Cos               3.821816e-01     0.618208     0.543425  -0.038026   

                                SMAPE  
Temperature                 56.497552  
Precipitation_accumulated    1.566134  
Humidity                    12.645638  
Wind_Speed_kmh              95.814017  
Soil

[I 2025-05-27 21:33:24,804] Trial 1 finished with value: 0.24141448681255467 and parameters: {'num_target_lags': 11, 'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 9, 'max_features': 0.7378961010099323, 'bootstrap': False, 'criterion': 'squared_error', 'min_weight_fraction_leaf': 0.2591903633753151}. Best is trial 0 with value: 0.17478106779814798.
Specified past encoders in `add_encoders` at model creation but model does not accept past covariates. past encoders will be ignored.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.


...

Starting time series verification for consistency...

Performance metrics:
                                    MSE         RMSE          MAE         R2  \
Temperature                3.757233e+01     6.129627     5.163034  -0.234918   
Precipitation_accumulated  4.240873e+01     6.512198     6.231010 -10.978842   
Humidity                   1.465470e+02    12.105662    10.585833  -0.002856   
Wind_Speed_kmh             3.314590e+00     1.820601     1.414636  -1.455572   
Soil_Moisture              5.345214e+06  2311.971980  2298.862625 -86.254468   
Soil_Temperature           3.237937e+01     5.690287     4.867265  -0.999223   
Wind_Dir_Sin               5.607651e-01     0.748843     0.691811  -0.022412   
Wind_Dir_Cos               1.516536e+00     1.231477     1.075259  -3.096975   

                                SMAPE  
Temperature                 60.041291  
Precipitation_accumulated    1.349878  
Humidity                    12.393060  
Wind_Speed_kmh             158.836165  
Soil

[I 2025-05-27 21:35:57,238] Trial 61 finished with value: 0.1378758720584885 and parameters: {'num_target_lags': 7, 'n_estimators': 100, 'max_depth': 17, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 0.520559248082299, 'bootstrap': False, 'criterion': 'squared_error', 'min_weight_fraction_leaf': 0.011879303548965563}. Best is trial 47 with value: 0.13107686673009888.
Specified past encoders in `add_encoders` at model creation but model does not accept past covariates. past encoders will be ignored.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.


Starting time series verification for consistency...

Performance metrics:
                                     MSE        RMSE         MAE        R2  \
Temperature                    24.523085    4.952079    3.921532  0.193692   
Precipitation_accumulated      31.153588    5.581540    4.958746 -7.787160   
Humidity                      165.419343   12.861545   10.108681 -0.132555   
Wind_Speed_kmh                  1.508882    1.228366    0.980227 -0.118492   
Soil_Moisture              463296.188449  680.658643  616.568075 -6.539721   
Soil_Temperature               16.697003    4.086197    3.176244 -0.030236   
Wind_Dir_Sin                    0.440715    0.663864    0.556833  0.195984   
Wind_Dir_Cos                    0.363394    0.602822    0.522788  0.018641   

                                SMAPE  
Temperature                 52.386558  
Precipitation_accumulated    1.072780  
Humidity                    11.846739  
Wind_Speed_kmh              81.080307  
Soil_Moisture         

[I 2025-05-27 21:35:59,293] Trial 62 finished with value: 0.12811281369865007 and parameters: {'num_target_lags': 5, 'n_estimators': 150, 'max_depth': 18, 'min_samples_split': 13, 'min_samples_leaf': 5, 'max_features': 0.5774519952255706, 'bootstrap': False, 'criterion': 'squared_error', 'min_weight_fraction_leaf': 0.0003584570012586107}. Best is trial 62 with value: 0.12811281369865007.


Starting time series verification for consistency...

Performance metrics:
                                     MSE        RMSE         MAE         R2  \
Temperature                    14.323032    3.784578    2.995731   0.529259   
Precipitation_accumulated      51.384389    7.168291    6.180002 -13.452453   
Humidity                      121.655713   11.029765    8.607518   0.166945   
Wind_Speed_kmh                  1.451366    1.204727    0.914026  -0.076568   
Soil_Moisture              447167.581840  668.705901  601.501976  -6.236364   
Soil_Temperature                7.267495    2.695829    1.955074   0.552490   
Wind_Dir_Sin                    0.455743    0.675087    0.552370   0.167811   
Wind_Dir_Cos                    0.372666    0.610464    0.523104  -0.005802   

                                SMAPE  
Temperature                 42.789317  
Precipitation_accumulated    1.339858  
Humidity                    10.222287  
Wind_Speed_kmh              79.418938  
Soil_Moisture

In [8]:
print(f"Best Value (Minimum): {study.best_value}")
print(f"Best Parameters: {study.best_params}")

Best Value (Minimum): 0.12811281369865007
Best Parameters: {'num_target_lags': 5, 'n_estimators': 150, 'max_depth': 18, 'min_samples_split': 13, 'min_samples_leaf': 5, 'max_features': 0.5774519952255706, 'bootstrap': False, 'criterion': 'squared_error', 'min_weight_fraction_leaf': 0.0003584570012586107}


In [9]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Best results saved in {json_path}")


Best results saved in optuna_iteration_metrics/best_trial.json
